# Convert COCO Segmentation Dataset to Vertex AI Segmentation
Contributor: michaelmenzel@google.com

In [ ]:
#@title Parameters & Authenticate
ANNOTATION_FILE = "gs://visual-inspection-demo-datasets-us-central1/car-damage-coco-segmentation-kaggle/train/COCO_train_annos.json" # @param {type:"string"}
TARGET_BUCKET = "gs://visual-inspection-demo-datasets-us-central1/car-damage-coco-segmentation-kaggle/train" # @param {type:"string"}



import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

In [ ]:
#@title Load Annotations
!gsutil cp $ANNOTATION_FILE .

import json
import os

with open(ANNOTATION_FILE.split(os.sep)[-1], 'r') as af:
  data = json.load(af)

Copying gs://visual-inspection-demo-datasets-us-central1/car-damage-coco-segmentation-kaggle/train/COCO_train_annos.json...
/ [1 files][192.3 KiB/192.3 KiB]                                                
Operation completed over 1 objects/192.3 KiB.                                    


In [ ]:
#@title Helpers
import io
import numpy as np

from PIL import Image

from google.cloud import storage


def get_gcs_image_size(image_url):
  """Get the size of an image in Google Cloud Storage.

  Args:
    image_url: The gs:// url of the image.

  Returns:
    A tuple containing the width and height of the image in pixels.
  """

  # Initialize client that will be used to send requests. This client only needs to be created
  # once, and can be reused for multiple requests.
  storage_client = storage.Client()

  # Get the image from the bucket.
  bucket = storage_client.bucket(image_url.split("//")[1].split("/")[0])
  blob = bucket.blob("/".join(image_url.split("//")[1].split("/")[1:]))
  image_bytes = blob.download_as_bytes()

  # Open the image using Pillow.
  image = Image.open(io.BytesIO(image_bytes))

  # Return the width and height of the image.
  return image.size

def construct_img_path(img):
  return os.path.join(os.path.dirname(ANNOTATION_FILE), img)

def clip_float(float_nbr, min_val=0, max_val=1):
  return max(min(float_nbr, max_val), min_val)

def format_float(float_nbr, precision=7):
  return float(np.format_float_positional(float_nbr, precision=precision))

In [ ]:
#@title Convert Annotations

from tqdm import tqdm

vai_annotations = []
image_annotations = {construct_img_path(img['file_name']): []
                     for img in data['images']}

for annot in data['annotations']:
  image = construct_img_path(data['images'][annot['image_id']]['file_name'])
  image_annotations[image].append(annot)

category_names = [cat['name'] for cat in data['categories']]

for img, annot in tqdm(image_annotations.items()):
  width, height = get_gcs_image_size(img)
  vai_annotations.append(
    {
        "imageGcsUri": img,
        "polygonAnnotations": [{
            "displayName": category_names[ann['category_id']-1],
            "vertexes": [{"x": clip_float(x / width),
                      "y": clip_float(y / height)}
                      for x, y in zip(ann['segmentation'][0][::2],
                                      ann['segmentation'][0][1::2])]
          } for ann in annot]
    })

vai_annotations

100%|██████████| 59/59 [00:34<00:00,  1.71it/s]


[{'imageGcsUri': 'gs://visual-inspection-demo-datasets-us-central1/car-damage-coco-segmentation-kaggle/train/10.jpg',
  'polygonAnnotations': [{'displayName': 'damage',
    'vertexes': [{'x': 0.3701171875, 'y': 0.2109375},
     {'x': 0.3291015625, 'y': 0.212890625},
     {'x': 0.29296875, 'y': 0.2177734375},
     {'x': 0.310546875, 'y': 0.232421875},
     {'x': 0.341796875, 'y': 0.2314453125},
     {'x': 0.3603515625, 'y': 0.2255859375},
     {'x': 0.3662109375, 'y': 0.2216796875}]},
   {'displayName': 'damage',
    'vertexes': [{'x': 0.28125, 'y': 0.224609375},
     {'x': 0.2578125, 'y': 0.232421875},
     {'x': 0.2392578125, 'y': 0.2431640625},
     {'x': 0.220703125, 'y': 0.2578125},
     {'x': 0.2353515625, 'y': 0.2578125},
     {'x': 0.2431640625, 'y': 0.2470703125},
     {'x': 0.2626953125, 'y': 0.2470703125},
     {'x': 0.2685546875, 'y': 0.244140625},
     {'x': 0.27734375, 'y': 0.232421875}]},
   {'displayName': 'damage',
    'vertexes': [{'x': 0.1572265625, 'y': 0.3232421875}

In [ ]:
#@title Store Annotations
with open('vertexai_car_damage_segmentation_polygon.jsonl', 'w') as of:
  for l in vai_annotations:
    json.dump(l, of)
    of.write('\n')

!gsutil cp 'vertexai_car_damage_segmentation_polygon.jsonl' $TARGET_BUCKET

Copying file://vertexai_car_damage_segmentation_polygon.jsonl [Content-Type=application/octet-stream]...
/ [1 files][109.6 KiB/109.6 KiB]                                                
Operation completed over 1 objects/109.6 KiB.                                    
